In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
# get the list of cities we kept from the reviews
yelpframe = pd.read_csv('./data/all_reviews_with_businesses_no_text.csv', index_col=[0])
phxframe = yelpframe[yelpframe['state'] == 'AZ'].copy()
phxframe['city'] = phxframe['city'].apply(lambda x: str(x).lower())
phxframe['city'] = phxframe['city'].apply(lambda x: x.strip(' '))
citycounts = phxframe['city'].value_counts()
citiestokeep = citycounts[(citycounts >= 1000)].index.tolist()

In [3]:
businessframe = pd.read_json('../labs_data/yelp_academic_dataset_business.json', lines=True, orient='records')

In [4]:
# filter businesses by said cities
phxbizframe = businessframe[businessframe['state'] == 'AZ'].copy()
phxbizframe['city'] = phxbizframe['city'].apply(lambda x: str(x).lower())
phxbizframe['city'] = phxbizframe['city'].apply(lambda x: x.strip(' '))
phxbizframe = phxbizframe[(phxbizframe['city'].isin(citiestokeep))]

In [5]:
# make binary feature for most common categories
phxbizframe['cat_list'] = phxbizframe['categories'].apply(lambda x: str(x).replace(' ', '').split(','))

# Binarise labels
mlb = MultiLabelBinarizer()
expanded_cat_data = mlb.fit_transform(phxbizframe['cat_list'])
cat_classes = ['category_' + x for x in mlb.classes_]

expanded_cats = pd.DataFrame(expanded_cat_data)
expanded_cats.columns = cat_classes

# get rid of category features with less than 1000 businesses
catcounts = expanded_cats.sum().sort_values(ascending=False)
catstodrop = catcounts[(catcounts < 1000)].index.tolist()
expanded_cats.drop(columns=catstodrop, inplace=True)

# add category features, remove unused features
phxbizframe.reset_index(drop=True, inplace=True)
concatenated = pd.concat([phxbizframe, expanded_cats], axis=1)
concatenated.drop(columns=['postal_code','latitude','longitude','is_open','attributes','cat_list','hours'], inplace=True)
concatenated.rename(columns={'stars':'aggregate_rating'}, inplace=True)

In [5]:
import psycopg2
from dotenv import load_dotenv
import os
load_dotenv()

True

In [6]:
# get postgres credentials
rds_credentials = {'AWS_RDS_HOST' : os.getenv('AWS_RDS_HOST'),
               'AWS_RDS_PORT' : os.getenv('AWS_RDS_PORT'),
               'AWS_RDS_USER' : os.getenv('AWS_RDS_USER'),
               'AWS_RDS_PASS' : os.getenv('AWS_RDS_PASS'),
               'AWS_RDS_DB' : os.getenv('AWS_RDS_DB')}

In [8]:
conn = psycopg2.connect(host=rds_credentials['AWS_RDS_HOST'],
                  database=rds_credentials['AWS_RDS_DB'],
                  user=rds_credentials['AWS_RDS_USER'],
                  password=rds_credentials['AWS_RDS_PASS'],
                  port=rds_credentials['AWS_RDS_PORT'])
cur = conn.cursor()

In [32]:
# create business data table
create_business_data_table = """
CREATE TABLE business_data(
business_id TEXT,
name TEXT,
address TEXT,
city TEXT,
state TEXT,
aggregate_rating FLOAT,
review_count INT,
categories TEXT,
category_ActiveLife INT,
category_American_New INT,
category_American_Traditional INT,
category_Apartments INT,
category_Arts_Entertainment INT,
category_AutoRepair INT,
category_Automotive INT,
category_Bars INT,
category_Beauty_Spas INT,
category_Breakfast_Brunch INT,
category_Burgers INT,
category_Coffee_Tea INT,
category_Contractors INT,
category_CosmeticDentists INT,
category_Dentists INT,
category_Doctors INT,
category_Education INT,
category_EventPlanning_Services INT,
category_Fashion INT,
category_FastFood INT,
category_FinancialServices INT,
category_Fitness_Instruction INT,
category_Food INT,
category_GeneralDentistry INT,
category_HairRemoval INT,
category_HairSalons INT,
category_Health_Medical INT,
category_Home_Garden INT,
category_HomeServices INT,
category_Hotels_Travel INT,
category_LocalServices INT,
category_Mexican INT,
category_NailSalons INT,
category_Nightlife INT,
category_Pets INT,
category_Pizza INT,
category_ProfessionalServices INT,
category_RealEstate INT,
category_Restaurants INT,
category_Sandwiches INT,
category_Shopping INT,
category_SkinCare INT
);
"""
# execute table creation
cur.execute("DROP TABLE IF EXISTS business_data")
cur.execute(create_business_data_table)
conn.commit()

In [33]:
insertion_query = """INSERT INTO business_data (business_id,
name,
address,
city,
state,
aggregate_rating,
review_count,
categories,
category_ActiveLife,
category_American_New,
category_American_Traditional,
category_Apartments,
category_Arts_Entertainment,
category_AutoRepair,
category_Automotive,
category_Bars,
category_Beauty_Spas,
category_Breakfast_Brunch,
category_Burgers,
category_Coffee_Tea,
category_Contractors,
category_CosmeticDentists,
category_Dentists,
category_Doctors,
category_Education,
category_EventPlanning_Services,
category_Fashion,
category_FastFood,
category_FinancialServices,
category_Fitness_Instruction,
category_Food,
category_GeneralDentistry,
category_HairRemoval,
category_HairSalons,
category_Health_Medical,
category_Home_Garden,
category_HomeServices,
category_Hotels_Travel,
category_LocalServices,
category_Mexican,
category_NailSalons,
category_Nightlife,
category_Pets,
category_Pizza,
category_ProfessionalServices,
category_RealEstate,
category_Restaurants,
category_Sandwiches,
category_Shopping,
category_SkinCare) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [34]:
business_tuples = tuple([tuple(x) for x in concatenated.itertuples(index=False)])

In [35]:
cur.executemany(insertion_query, business_tuples)

In [36]:
conn.commit()

In [9]:
conn.close()

In [42]:
conn.close()